In [ ]:
import os

os.environ["OPENAI_API_KEY"] = (
    "--------------------------------------------------"
)

In [2]:
from langchain_community.tools import BraveSearch
from langchain_openai import ChatOpenAI

from langchain_core.tools import tool


In [ ]:
# step 1 -> setup Brave Serach tool
api_key = "--------------------------------------------------"

brave_tool = BraveSearch.from_api_key(api_key=api_key, search_kwargs={"count": 20})

In [4]:
# step 2 : calling LLM model

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=512, n=4)

### GET TRENDING LEAGUE FOR HOME PAGE

In [5]:
# define tool for get-Trending-Cricket-League


@tool
def brave_cricket_search(_input: str = "") -> str:
    """
    Performs a globally refined Brave search for currently live or ongoing cricket leagues and tours.
    Filters out previews, news, and non-authoritative content.
    """
    query = (
        '"live cricket matches" OR '
        '"ongoing domestic cricket matches India" OR '
        '"current international cricket tours" '
        '"current state-level cricket tournaments and League India" '
        "-preview -opinion "
        "site:cricbuzz.com OR site:espncricinfo.com OR site:icc-cricket.com"
    )
    return brave_tool.run(query)

In [6]:
search_snippets = brave_cricket_search.invoke("")

search_snippets

'[{"title": "Cricket Fixtures: List of Ongoing, Upcoming and Concluded Series", "link": "https://www.espncricinfo.com/cricket-fixtures", "snippet": "Find the list of <strong>domestic</strong> <strong>and</strong> <strong>international</strong> <strong>current</strong> <strong>cricket</strong> series, upcoming series, and recently concluded <strong>tournaments</strong> on our comprehensive <strong>cricket</strong> fixtures page. Current Cricket \\u00b7 Future Series / Tournaments \\u00b7 Recently Concluded \\u00b7 Download Schedule \\u00b7 Download ICC\'s Future tours programme (Men) Download ICC\'s Future tours programme (Women) International Tours \\u00b7 England vs IndiaJun 13, 2025 - Jul 15, 2025 \\u00b7 Fixtures & ResultsSquads \\u00b7"}, {"title": "ICC Cricket Fixtures & Results | ICC", "link": "https://www.icc-cricket.com/fixtures-results", "snippet": "Stay updated with the latest ICC <strong>cricket</strong> fixtures, results, <strong>match</strong> schedules, and scores. Explor

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [8]:
# define output Format


class LeagueOutputFormat(BaseModel):
    trending_leagues_and_tours: List[str] = Field(
        ...,
        description="List of currently ongoing cricket leagues and international tours",
    )

In [9]:
# create a paerser

output_parser = PydanticOutputParser(pydantic_object=LeagueOutputFormat)

In [10]:
# ==== Function : get-Trending-Cricket-League ====


def generate_cricket_prompt(search_snippets: str) -> str:
    prompt = ChatPromptTemplate.from_template(
        """
You are a cricket data assistant.

From the following web search result snippets, extract the names of all **currently ongoing or live cricket leagues, tournaments, or international tours**.

🎯 Your goal is to identify **active** events happening **right now** across:
- ✅ International level (e.g., "India Tour of England", "ICC World Test Championship")
- ✅ Domestic leagues (e.g., "Vitality Blast", "CPL", "BBL", "PSL")
- ✅ Indian domestic cricket (e.g., "Ranji Trophy", "Duleep Trophy", "Syed Mushtaq Ali Trophy", "Vijay Hazare Trophy", "Deodhar Trophy", "Irani Cup", "CK Nayudu Trophy", "Women's T20 Trophy")
- ✅ National-level tournaments or bilateral series (e.g., "Sri Lanka vs Bangladesh", "India Women vs South Africa Women")

✅ Include:
- Any league, tournament, or tour clearly mentioned as **live**, **in progress**, **ongoing**, or currently providing **live scores**
- Competitions actively taking place in **any country**, especially **India’s domestic cricket** (Men’s or Women’s cricket)

🚫 Exclude:
- Future or upcoming events (e.g., "IPL 2026", "India Tour of Australia 2026")
- Past or recently concluded series
- Match previews or one-off matches (e.g., "2nd T20 preview", "Final match preview")
- Rankings, scorecards, or statistical pages

📦 Return the data in **pure JSON format only** (no explanation or comments) as:

{format_instructions}

📄 Search Snippets:
{search_snippets}
"""
    )

    formatted_prompt = prompt.format(
        search_snippets=search_snippets,
        format_instructions=output_parser.get_format_instructions(),
    )

    return formatted_prompt

In [11]:
# Run the LLM


def extract_ongoing_cricket_leagues(search_snippets: str):
    prompt = generate_cricket_prompt(search_snippets)
    response = llm.invoke(prompt).content
    try:
        return output_parser.parse(response)
    except Exception as e:
        return {"error": "Failed to parse response", "raw_output": response}

In [12]:
homwPageChips = extract_ongoing_cricket_leagues(search_snippets)

In [13]:
homwPageChips

LeagueOutputFormat(trending_leagues_and_tours=['England vs India', 'ICC World Test Championship, 2025-2027', 'Vitality Blast Women', 'Vitality Blast Men', 'CWC Challenge League Group B', 'CWC Challenge League Group A', "Women's Championship", 'Major League Cricket', 'India Women vs South Africa Women', 'Sri Lanka vs Bangladesh', 'West Indies vs Australia', 'ENG Women vs IND Women'])

### STUMPZY Chat Starts

In [14]:
@tool
def brave_cricket_search(_input: str = "") -> str:
    """
    Performs a globally refined Brave search for currently live or ongoing cricket leagues and tours.
    Filters out previews, news, and non-authoritative content.
    """
    query = (
       'England vs India live score'
    )
    return brave_tool.run(query)

In [15]:
search_snippets = brave_cricket_search.invoke("")

search_snippets

'[{"title": "IND U19 vs ENG U19, 5th ODI Highlights: England win final game by 7 wickets, but India seal series 3-2 | Cricket News - The Indian Express", "link": "https://indianexpress.com/article/sports/cricket/ind-u19-vs-eng-u19-5th-youth-odi-live-cricket-score-india-england-under-19-worcester-today-updates-vaibhav-suryavanshi-thomas-rew-10111130/", "snippet": "The Indian Express website has been rated GREEN for its credibility and trustworthiness by Newsguard, a global service that rates news sources for their journalistic standards. Stay updated with the latest sports news across Cricket, Football, Chess, and more. Catch all the action with real-time <strong>live</strong> cricket <strong>score</strong> ... The Indian Express website has been rated GREEN for its credibility and trustworthiness by Newsguard, a global service that rates news sources for their journalistic standards. Stay updated with the latest sports news across Cricket, Football, Chess, and more. Catch all the actio

In [21]:
from langchain_tavily import TavilySearch


tavily = TavilySearch(
    max_results=30,            # Increase number of results
    topic="general",
    include_answer=True,       # Provide direct answer summaries
    include_raw_content=False, 
    include_images=False,
    search_depth="advanced",   # Deeper crawling for context
    time_range="day",          # Limit to last 24 hours
    include_domains=[
        "cricbuzz.com", "espncricinfo.com", "icc-cricket.com"
    ],
    exclude_domains=None
)

In [22]:
# tool.invoke({"query": "What happened at the last wimbledon"})

In [23]:
resp = tavily.invoke({
    "query": (
        '"live cricket match" OR '
        '"today cricket schedule" OR '
        '"ongoing cricket series" OR '
        '"domestic cricket tournament India" OR '
        '"international cricket fixtures today" OR '
        '"T20 league live score"'
    ),
    "include_domains": None,  # already set
    "exclude_domains": None
})



In [25]:
resp


{'query': '"live cricket match" OR "today cricket schedule" OR "ongoing cricket series" OR "domestic cricket tournament India" OR "international cricket fixtures today" OR "T20 league live score"',
 'follow_up_questions': None,
 'answer': "Today's international cricket matches include Sri Lanka vs Bangladesh in the 3rd ODI. The India tour of England 2025 is scheduled. Live scores for Bangladesh tour of Sri Lanka 2025 are available.",
 'images': [],
 'results': [{'url': 'https://www.cricbuzz.com/cricket-schedule/upcoming-series/all',
   'title': 'Cricket Schedule - International, domestic and T20 matches',
   'content': 'Schedule of International, T20 League, Indian, Australian and English domestic cricket matches on Cricbuzz.',
   'score': 0.55623144,
   'raw_content': None},
  {'url': 'https://www.cricbuzz.com/cricket-match/live-scores/upcoming-matches',
   'title': 'India tour of England 2025 | Live Cricket Score | Scorecard',
   'content': 'Get Live Cricket Score, Ball by Ball Comme